In [1]:
#pip install rpy2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
 !pip install pyreadr

In [4]:
import os
import numpy as np
import pandas as pd
import pyreadr as py 
import tensorflow as tf
import gc 
import itertools
from sklearn.utils import shuffle

gc.collect() 




0

Load RData as Pandas DataFrame

In [5]:
a1 = py.read_r("drive/MyDrive/TEP_dataset/TEP_FaultFree_Training.RData")
fault_free_training = a1['fault_free_training']
a1 = None 
a2 = py.read_r("drive/MyDrive/TEP_dataset/TEP_Faulty_Training.RData")
faulty_training = a2['faulty_training']
a2 = None
a3 = py.read_r("drive/MyDrive/TEP_dataset/TEP_FaultFree_Testing.RData")
fault_free_testing = a3['fault_free_testing']
a3 = None 
a4 = py.read_r("drive/MyDrive/TEP_dataset/TEP_Faulty_Testing.RData")
faulty_testing = a4['faulty_testing']
a4 = None 

In [6]:
gc.collect()

68

Get Data Stream Iterator

In [7]:
def get_input_fn(data_set, num_epochs = None, shuffle=True):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = data_set.drop(['faultNumber', 'simulationRun', 'sample'], axis=1),
        y = data_set['faultNumber'],
        num_epochs = num_epochs,
        shuffle = shuffle
        )

def get_feature_cols(data_set):
    return [tf.feature_column.numeric_column(key=k)
           for k in data_set.drop(['faultNumber', 'simulationRun', 'sample'], axis=1).keys()]

Simple Neural Network

In [8]:
train_df = faulty_training.copy(deep=False)
test_df = faulty_testing.copy(deep=False)

In [35]:
regressor = tf.estimator.DNNRegressor(feature_columns=get_feature_cols(train_df), hidden_units=[20, 20])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp40_t2wf7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
regressor.train(get_input_fn(train_df, 10), steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp40_t2wf7/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 52419.69, step = 0
INFO:tensorflow:global_step/sec: 60.7597
INFO:tensorflow:loss = 105.44116, step = 100 (1.649 sec)
INFO:tensorflow:global_step/sec: 94.768
INFO:tensorflow:loss = 8.168355, step = 200 (1.054 sec)
INFO:tensorflow:global_step/sec: 96.3569
INFO:tensorflow:loss = 24.307144, step = 300 (1.040 sec)
INFO:tensorflow:global_step/sec: 90.3152
INFO:tensorflow:loss = 54.23652, step = 400 (1.106 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 500...
INFO:tensorflow:Saving checkpoi

In [37]:
y = regressor.predict(input_fn = get_input_fn(test_df[:10], num_epochs=1, shuffle=False))
predictions = list(p["predictions"] for p in itertools.islice(y, 10))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp40_t2wf7/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [38]:
print ("Prediction:" + str([int(i[0]) for i in predictions]))
print ("Actual:" + str(test_df['faultNumber'][:10].values))

Prediction:[8, 9, 9, 9, 9, 8, 9, 9, 9, 9]
Actual:[1 1 1 1 1 1 1 1 1 1]
